In [2]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.tree import DecisionTreeClassifier
import pickle
import numpy as np

In [4]:
# Veri setini yüklemek
df = pd.read_csv("Training.csv")

# Özellikler ve hedef değişkeni ayırma
X = df.iloc[:, :-1]  # Hedef değişken hariç tüm sütunlar
y = df["prognosis"]  # Hedef değişken

 # 'prognosis' sütununu X'ten çıkarma
X = X.drop('prognosis', axis=1) 

In [5]:
# LabelEncoder nesnesini oluştur ve hedef değişkeni dönüştür
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Veriyi eğitim ve test setlerine ayırma
X_train, X_test, y_train, y_test = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42
) 

In [6]:
sc = StandardScaler()
X_train=sc.fit_transform(X_train)
X_test=sc.transform(X_test)


In [7]:
def modeller(model):
    model.fit(X_train,y_train)
    tahmin=model.predict(X_test)
    skor=accuracy_score(y_test,tahmin)
    return round(skor*100,2)

print(modeller(DecisionTreeClassifier()))

In [8]:

# RandomForestClassifier modelini oluşturma
# Burada bazı parametreleri ayarlayarak model karmaşıklığını kontrol edebilirsiniz.
model = RandomForestClassifier(n_estimators=100, max_depth=None, random_state=42)

# K-fold çapraz doğrulama ile modelin genelleştirme performansını değerlendirme
cv_scores = cross_val_score(model, X_train, y_train, cv=5)
print(f"Çapraz doğrulama skorları: {cv_scores}")
print(f"Ortalama çapraz doğrulama doğruluk: {cv_scores.mean()}")

Çapraz doğrulama skorları: [1. 1. 1. 1. 1.]
Ortalama çapraz doğrulama doğruluk: 1.0


In [10]:
 # Hiperparametre aralıklarını belirleme
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_depth': [10, 20, None],
    'min_samples_split': [2, 4, 6]
}

# GridSearchCV ile hiperparametre optimizasyonu
grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)

# En iyi parametreleri ve bunların skorlarını yazdırma
print("En iyi parametreler:", grid_search.best_params_)
print("En iyi skor:", grid_search.best_score_)


Fitting 5 folds for each of 27 candidates, totalling 135 fits
En iyi parametreler: {'max_depth': 10, 'min_samples_split': 2, 'n_estimators': 100}
En iyi skor: 1.0


In [11]:
# Test setini yüklemek
df_test = pd.read_csv("Testing.csv")

# Özellikler ve hedef değişkeni ayırma
X_test = df_test.iloc[:, :-1]  # Hedef değişken hariç tüm sütunlar
y_test = df_test["prognosis"]  # Hedef değişken

# Hedef değişkeni sayısal değerlere dönüştür
y_test_encoded = label_encoder.transform(y_test)

# GridSearchCV ile bulunan en iyi parametrelerle modeli yeniden oluşturma
best_params = grid_search.best_params_
model_best = RandomForestClassifier(**best_params, random_state=42)

# Modeli eğitim seti ile eğitme
model_best.fit(X_train, y_train)

# Modeli test seti üzerinde değerlendirme
y_pred = model_best.predict(X_test)
accuracy = accuracy_score(y_test_encoded, y_pred)
print(f"Test seti üzerindeki modelin doğruluk oranı: {accuracy}")


Test seti üzerindeki modelin doğruluk oranı: 0.9761904761904762


In [13]:

# Modelin F1 skorunu hesaplama
f1 = f1_score(y_test_encoded, y_pred, average='weighted')
print(f"F1 Skoru: {f1}")

F1 Skoru: 0.9761904761904762


In [15]:

# Modeli 'model.pkl' olarak kaydetme
with open('model.pkl', 'wb') as file:
    pickle.dump(model_best, file)

print("Model 'model.pkl' olarak kaydedildi.")

Model 'model.pkl' olarak kaydedildi.


In [21]:
importances = model_best.feature_importances_
feature_names = X_train.columns
feature_importances = sorted(zip(importances, feature_names), reverse=True)
